In [1]:
import requests
from urllib.request import urlopen
from bs4 import BeautifulSoup

import pandas as pd

import re
from datetime import datetime, timedelta
import time

In [2]:
query = '백신'
url = "https://search.naver.com/search.naver?where=news&query=" + query

In [3]:
web = requests.get(url).content
source = BeautifulSoup(web, 'html.parser')

In [4]:
urls_list = []

for urls in source.find_all('a', {'class' : "info"}):
    if urls.attrs["href"].startswith("https://news.naver.com"): # starts with 사용
        urls_list.append(urls.attrs["href"])

urls_list

['https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=102&oid=001&aid=0012925037',
 'https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=102&oid=003&aid=0010950396',
 'https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=104&oid=437&aid=0000286879',
 'https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=105&oid=366&aid=0000787929',
 'https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=101&oid=277&aid=0005032174',
 'https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=104&oid=015&aid=0004654199',
 'https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=102&oid=421&aid=0005851186',
 'https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=104&oid=079&aid=0003598376',
 'https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=102&oid=057&aid=0001636295',
 'https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=104&oid=052&aid=0001690941']

In [5]:
title_list = []

for news_titles in source.find_all('a', {'class' : 'news_tit'}):
    title_list.append(news_titles.get_text())

title_list

['백신 부작용 학생에 최대 500만원 지원…심리위기엔 600만원',
 "'30만원 이하' 백신 피해보상, 지자체가 결정한다",
 '이스라엘 연구진 "백신 4차 접종, 오미크론에 효과 크지 않아"',
 '국산 코로나19 백신 9부 능선 넘었다…임상 3상 참여자 모집 완료',
 "SK바사 코로나19 백신 임상 3상 8부 능선… '오미크론 효능'은",
 '모더나 CEO "독감·코로나 한번에 예방하는 백신 개발"',
 '백신 이상반응 청소년, 인과성 인정 안돼도 최대 500만원 지원',
 '"백신 4차접종 후 항체, 오미크론 예방에 불충분"',
 '당국 "임산부, 백신 접종 권고 대상…방역패스 예외 아냐"',
 '조코비치, 프랑스오픈도 못 가나..."백신 맞아야 출전 가능"']

In [6]:
# 크롬브라우저에서 보내는 요청으로 인식하도록 HTTP Request에 Header 정보 추가
headers = {'User-Agent':'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.132 Safari/537.36'}

web_news = requests.get(urls_list[0], headers=headers).content
source_news = BeautifulSoup(web_news, 'html.parser')

In [7]:
title = source_news.find('h3', {'id' : 'articleTitle'}).get_text()

title

'백신 부작용 학생에 최대 500만원 지원…심리위기엔 600만원'

In [8]:
date = source_news.find('span', {'class' : 't11'}).get_text().replace(" ","") # 빈칸 제거

date1 = date[:11] # 날짜까지 파싱
date2 = date[13:] # 시간부분 파싱
date3 = (lambda x : 'am' if x == '오전' else 'pm')(date[11:13]) # 오전 -> am, 오후 -> pm 수정
date4 = date1 + date2 + date3

date = pd.Timestamp(date4)

date

Timestamp('2022-01-18 11:21:00')

In [9]:
article = source_news.find('div', {'id' : 'articleBodyContents'}).get_text()
article = article.replace("\n", "") 
article = article.replace("// flash 오류를 우회하기 위한 함수 추가function _flash_removeCallback() {}", "") 
article = article.replace("동영상 뉴스       ", "")
article = article.replace("동영상 뉴스", "")
article = article.strip()

article

"이상반응 신고율 0.27%…인과성 인정 어려워 국가보상 못받는 학생 지원정신건강 관심군 4.6%·극단선택 학생↑…24시간 상담·전문 의료서비스코로나19 백신접종 받는 학생들[연합뉴스 자료사진]    (서울=연합뉴스) 김지연 기자 = 코로나19 유행이 장기화하는 가운데 정부가 극심한 불안, 우울에 시달리는 학생들의 심리 회복과 백신 중증 이상반응을 겪는 학생들의 치료에 각각 최대 600만원, 500만원을 지원한다.    유은혜 부총리 겸 교육부 장관은 18일 정부세종청사에서 이같은 내용의 '코로나19 극복을 위한 학생 건강회복 지원방안'을 발표했다.  백신 이상반응 신고율 0.27%…국가보상 못 받는 경우 지원    접종 당시 만 18세 이하 학생 중 접종 이후 90일 내 중증 이상반응이 발생했으나 인과성이 인정되기 어려워 국가 보상을 받지 못할 때 보완적 의료비가 500만원 내로 지원된다.    '중증'은 증상 유형과 관계없이 국가보상 신청 액수가 본인부담금 기준 30만 원 이상일 경우이며, 국가 보상제도에 따른 예방접종피해보상 전문위원회 심의에서 기각된 사례에 해당한다.    교육부는 국내 백신접종 이상반응 의심사례가 접종 후 72일까지 신고되고 있는 점 등을 고려해 기간을 접종 후 90일 내로 정했다고 설명했다.    17일 기준 13∼18세 접종률은 1차 78.6%, 2차 66.5%다.    1, 2차를 통틀어 전체 402만1천208건 중 이상반응 의심사례는 1만915건(0.27%) 신고됐으며 그중 사망이나 아나필락시스 의심, 중환자실 입원, 영구장애 등 중대한 이상반응 신고 건수는 284건이다.    본인이나 보호자가 교육부가 지정한 위탁기관인 한국교육환경보호원에 의료비 영수증을 포함한 신청서를 제출하면 된다.    개인별 총액 500만 원 한도이나 교육급여(중위소득 50% 이하) 대상자 중 의료비 총액이 500만 원을 초과하면 최대 1천만 원 한도에서 지원이 가능하다.    국가보상제도 신청부터 통보까지 소요되는 기간이 최장 120일임을 고려해 사업

In [10]:
press_company = source_news.find('address', {'class' : 'address_cp'}).find('a').get_text() # 상위 class에서 'a' 태그 찾아오기

press_company

'연합뉴스'

In [11]:
query = '백신'

url = "https://search.naver.com/search.naver?where=news&query=" + query \
    + "&%2Ca%3Aall"

url

'https://search.naver.com/search.naver?where=news&query=백신&%2Ca%3Aall'

### '백신'으로 검색한 기사 5페이지 내용 가져오기

In [12]:
max_page = 5 # 5페이지까지 설정
query = '백신'

titles = []
dates = []
articles = []
article_urls = []
press_companies = []

In [13]:
# all 자리에 &start= 추가, 페이지수 설정
# str(current_call) => 1페이지 : 1 , 2페이지 : 11, 3페이지: 21, 4페이지: 31, 5페이지 : 41

current_call = 1
last_call = (max_page - 1) * 10 + 1 # 5페이지 = 41

while current_call <= last_call:
    
    print('\n{}번~{}번 기사 크롤링'.format(current_call, current_call+9))
    
    url = "https://search.naver.com/search.naver?where=news&query=" + query \
        + "&%2Ca%3A&start=" + str(current_call)

    # web, source 재설정
    web = requests.get(url).content
    source = BeautifulSoup(web, 'html.parser')

    urls_list = []
    for urls in source.find_all('a', {'class' : "info"}):
        if urls["href"].startswith("https://news.naver.com"):
            urls_list.append(urls["href"])

    for url in urls_list:
        try:
            # 네이버 접속 차단 우회
            headers = {'User-Agent':'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.132 Safari/537.36'}
            web_news = requests.get(url, headers=headers).content
            source_news = BeautifulSoup(web_news, 'html.parser')
            
            # 기사 제목
            title = source_news.find('h3', {'id' : 'articleTitle'}).get_text()
            print('Processing article : {}'.format(title))
            
            # 기사 발행 날짜
            date = source_news.find('span', {'class' : 't11'}).get_text()
            
            # 기사 본문, 수정
            article = source_news.find('div', {'id' : 'articleBodyContents'}).get_text()
            article = article.replace("\n", "")
            article = article.replace("// flash 오류를 우회하기 위한 함수 추가function _flash_removeCallback() {}", "")
            article = article.replace("동영상 뉴스       ", "")
            article = article.replace("동영상 뉴스", "")
            article = article.strip()
            
            # 발행 언론사
            press_company = source_news.find('address', {'class' : 'address_cp'}).find('a').get_text()
            
            # 빈 리스트에 추가
            titles.append(title)
            dates.append(date)
            articles.append(article)
            press_companies.append(press_company)
            article_urls.append(url)
        except:
            print('*** 다음 링크의 뉴스를 크롤링하는 중 에러가 발생했습니다 : {}'.format(url))
            
    # 대량의 데이터 => 쉬어주는 타이밍 삽입
    time.sleep(4)
    
    current_call += 10


1번~10번 기사 크롤링
Processing article : 백신 부작용 학생에 최대 500만원 지원…심리위기엔 600만원
Processing article : '30만원 이하' 백신 피해보상, 지자체가 결정한다
Processing article : 이스라엘 연구진 "백신 4차 접종, 오미크론에 효과 크지 않아"
Processing article : 국산 코로나19 백신 9부 능선 넘었다…임상 3상 참여자 모집 완료
Processing article : SK바사 코로나19 백신 임상 3상 8부 능선… '오미크론 효능'은
Processing article : 모더나 CEO "독감·코로나 한번에 예방하는 백신 개발"
Processing article : 백신 이상반응 청소년, 인과성 인정 안돼도 최대 500만원 지원
Processing article : "백신 4차접종 후 항체, 오미크론 예방에 불충분"
Processing article : 당국 "임산부, 백신 접종 권고 대상…방역패스 예외 아냐"
Processing article : 조코비치, 프랑스오픈도 못 가나..."백신 맞아야 출전 가능"

11번~20번 기사 크롤링
Processing article : 교육부 "청소년 코로나우울·백신접종 이상반응 등 의료비 지원"
Processing article : SK바이오 코로나 백신 임상 3상 대상자 4천여명 모집 완료
Processing article : 코로나·독감 동시 예방 백신 내년 가을에 나온다, 모더나가 개발중
Processing article : B세포림프종 환자, 백신접종 후 길랑바레 증후군 발생
Processing article : "믿을 건 백신 뿐"...'백신 접종 의무화' 잇따라
Processing article : 대형마트에서 장 보는 코로나19 백신 미접종자
Processing article : 화이자 백신 77만회분 19일 온다… 올해 누적 169.4만회분
Processing article : 청소년 백신 중대 이상반응 284건… 교

In [14]:
# 전체 데이터를 DataFrame으로 생성
article_df_vaccine = pd.DataFrame({'Title':titles, 
                           'Date':dates, 
                           'Article':articles, 
                           'URL':article_urls, 
                           'PressCompany':press_companies})

# result_연도월일_시분.csv 로 파일 생성
article_df_vaccine.to_excel('./data/result_vaccine_{}.xlsx'.format(datetime.now().strftime('%y%m%d_%H%M')), index=False, encoding='utf-8')

article_df_vaccine

,Title,Date,Article,URL,PressCompany
0,백신 부작용 학생에 최대 500만원 지원…심리위기엔 600만원,2022.01.18. 오전 11:21,이상반응 신고율 0.27%…인과성 인정 어려워 국가보상 못받는 학생 지원정신건강 관...,https://news.naver.com/main/read.naver?mode=LS...,연합뉴스
1,"'30만원 이하' 백신 피해보상, 지자체가 결정한다",2022.01.18. 오전 10:00,기사내용 요약피해보상금 30만원 이하인 경우시·도에서 인과성 심의→보상 결정[세종=...,https://news.naver.com/main/read.naver?mode=LS...,뉴시스
2,"이스라엘 연구진 ""백신 4차 접종, 오미크론에 효과 크지 않아""",2022.01.18. 오전 7:11,이스라엘의 한 여성이 코로나19 백신을 접종받고 있는 모습. 〈사진-로이터·연합뉴...,https://news.naver.com/main/read.naver?mode=LS...,JTBC
3,국산 코로나19 백신 9부 능선 넘었다…임상 3상 참여자 모집 완료,2022.01.18. 오전 10:31,"코로나19 백신·치료제 TF 회의 한국 , 태국, 필리핀 등 6개국 약 4000여...",https://news.naver.com/main/read.naver?mode=LS...,조선비즈
4,SK바사 코로나19 백신 임상 3상 8부 능선… '오미크론 효능'은,2022.01.18. 오전 11:42,대상자 4037명 모집 완료올 상반기 중 허가 목표초기 대응용… 추가 개발 시급SK...,https://news.naver.com/main/read.naver?mode=LS...,아시아경제신문
5,"모더나 CEO ""독감·코로나 한번에 예방하는 백신 개발""",2022.01.18. 오전 11:29,"""내년 하반기까지 개발 목표""""두세 번 맞는 번거로움 없애겠다""미국 제약회사 모더나...",https://news.naver.com/main/read.naver?mode=LS...,한경닷컴
6,"백신 이상반응 청소년, 인과성 인정 안돼도 최대 500만원 지원",2022.01.18. 오전 11:20,국가보상 받지 못한 중증 이상반응 의료비 보완다른 이유에 의한 이상반응 가능성 높아...,https://news.naver.com/main/read.naver?mode=LS...,뉴스1
7,"""백신 4차접종 후 항체, 오미크론 예방에 불충분""",2022.01.18. 오전 10:23,이스라엘 세계 첫 연구결과…병원 의료진 대상이스라엘의 한 고령자가 코로나19 백신 ...,https://news.naver.com/main/read.naver?mode=LS...,노컷뉴스
8,"당국 ""임산부, 백신 접종 권고 대상…방역패스 예외 아냐""",2022.01.18. 오후 4:17,"""미접종 임산부 사망 사례 있어""구체적 내용 목요일 발표 예정기사 내용과 무관한 참...",https://news.naver.com/main/read.naver?mode=LS...,MBN
9,"조코비치, 프랑스오픈도 못 가나...""백신 맞아야 출전 가능""",2022.01.18. 오전 2:16,남자 테니스 단식 세계 랭킹 1위 노바크 조코비치가 코로나19 백신을 맞지 않으면 ...,https://news.naver.com/main/read.naver?mode=LS...,YTN


### '코로나'로 검색한 기사 5페이지 내용 가져오기

In [15]:
max_page = 5 # 5페이지까지 설정
query = '코로나'

titles = []
dates = []
articles = []
article_urls = []
press_companies = []

In [16]:
# all 자리에 &start= 추가, 페이지수 설정
# str(current_call) => 1페이지 : 1 , 2페이지 : 11, 3페이지: 21, 4페이지: 31, 5페이지 : 41

current_call = 1
last_call = (max_page - 1) * 10 + 1 # 5페이지 = 41

while current_call <= last_call:
    
    print('\n{}번~{}번 기사 크롤링'.format(current_call, current_call+9))
    
    url = "https://search.naver.com/search.naver?where=news&query=" + query \
        + "&%2Ca%3A&start=" + str(current_call)

    # web, source 재설정
    web = requests.get(url).content
    source = BeautifulSoup(web, 'html.parser')

    urls_list = []
    for urls in source.find_all('a', {'class' : "info"}):
        if urls["href"].startswith("https://news.naver.com"):
            urls_list.append(urls["href"])

    for url in urls_list:
        try:
            # 네이버 접속 차단 우회
            headers = {'User-Agent':'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.132 Safari/537.36'}
            web_news = requests.get(url, headers=headers).content
            source_news = BeautifulSoup(web_news, 'html.parser')
            
            # 기사 제목
            title = source_news.find('h3', {'id' : 'articleTitle'}).get_text()
            print('Processing article : {}'.format(title))
            
            # 기사 발행 날짜
            date = source_news.find('span', {'class' : 't11'}).get_text()
            
            # 기사 본문, 수정
            article = source_news.find('div', {'id' : 'articleBodyContents'}).get_text()
            article = article.replace("\n", "")
            article = article.replace("// flash 오류를 우회하기 위한 함수 추가function _flash_removeCallback() {}", "")
            article = article.replace("동영상 뉴스       ", "")
            article = article.replace("동영상 뉴스", "")
            article = article.strip()
            
            # 발행 언론사
            press_company = source_news.find('address', {'class' : 'address_cp'}).find('a').get_text()
            
            # 빈 리스트에 추가
            titles.append(title)
            dates.append(date)
            articles.append(article)
            press_companies.append(press_company)
            article_urls.append(url)
        except:
            print('*** 다음 링크의 뉴스를 크롤링하는 중 에러가 발생했습니다 : {}'.format(url))
            
    # 대량의 데이터 => 쉬어주는 타이밍 삽입
    time.sleep(4)
    
    current_call += 10


1번~10번 기사 크롤링
Processing article : 코로나19 백신 중증이상반응 학생에 최대 1000만원 지원
Processing article : 국산 코로나19 백신 9부 능선 넘었다…임상 3상 참여자 모집 완료
Processing article : SK바사 코로나19 백신 임상 3상 8부 능선… '오미크론 효능'은
Processing article : 코로나 집밥에 즉석조리식품 시장 2조 '훌쩍'…해외서도 '인기'
Processing article : 코로나 시대, 중고생 희망직업 '컴퓨터공학자·개발자' 뜬다
*** 다음 링크의 뉴스를 크롤링하는 중 에러가 발생했습니다 : https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=106&oid=052&aid=0001691158
Processing article : 문 대통령, 코로나19 현장 종사자 등 1만 5천명에 설 선물
Processing article : 모더나 CEO "독감·코로나 한번에 예방하는 백신 개발"
Processing article : 광주시, 코로나19 예방접종 피해보상 신속 추진
Processing article : 코로나19 확진자 평균 치료비 308만원, 건보공단 부담 263만원

11번~20번 기사 크롤링
Processing article : "홍콩서 동물-사람 간 코로나19 전염 의심 사례 나와"
Processing article : 코로나 5차 접종 필요? "4차 접종으로 오미크론 못막아" 연구 결과 나왔다
Processing article : 코로나19 신규 확진자 4072명…누적 확진 70만명 넘어
Processing article : “터널 끝 빛 보인다”…WHO 전문가, 영국 코로나 종식 기대
Processing article : 김기현 “소상공인 코로나 지원 1000만원까지… 밀실추경 안돼”
Processing article : 코로나 확진자 치료비 2년간 8691억…86% 건보서 부담
*** 다음 링

In [17]:
# 전체 데이터를 DataFrame으로 생성
article_df_covid = pd.DataFrame({'Title':titles, 
                           'Date':dates, 
                           'Article':articles, 
                           'URL':article_urls, 
                           'PressCompany':press_companies})

# result_연도월일_시분.csv 로 파일 생성
article_df_covid.to_excel('./data/result_covid_{}.xlsx'.format(datetime.now().strftime('%y%m%d_%H%M')), index=False, encoding='utf-8')

article_df_covid

,Title,Date,Article,URL,PressCompany
0,코로나19 백신 중증이상반응 학생에 최대 1000만원 지원,2022.01.18. 오전 11:20,"기사내용 요약내달부터 내년 5월까지…최대 800명 혜택500만원 한도로, 교육급여대...",https://news.naver.com/main/read.naver?mode=LS...,뉴시스
1,국산 코로나19 백신 9부 능선 넘었다…임상 3상 참여자 모집 완료,2022.01.18. 오전 10:31,"코로나19 백신·치료제 TF 회의 한국 , 태국, 필리핀 등 6개국 약 4000여...",https://news.naver.com/main/read.naver?mode=LS...,조선비즈
2,SK바사 코로나19 백신 임상 3상 8부 능선… '오미크론 효능'은,2022.01.18. 오전 11:42,대상자 4037명 모집 완료올 상반기 중 허가 목표초기 대응용… 추가 개발 시급SK...,https://news.naver.com/main/read.naver?mode=LS...,아시아경제신문
3,코로나 집밥에 즉석조리식품 시장 2조 '훌쩍'…해외서도 '인기',2022.01.18. 오후 12:23,"농식품부·aT, 가공식품 세분시장 현황 보고서밀키트 등 가정간편식 인기즉석밥·떡볶이...",https://news.naver.com/main/read.naver?mode=LS...,financial news
4,"코로나 시대, 중고생 희망직업 '컴퓨터공학자·개발자' 뜬다",2022.01.18. 오후 12:00,교육부·직능원 진로교육 현황조사…중 8위·고 4위로 상승중고생 1위는 15년째 교사...,https://news.naver.com/main/read.naver?mode=LS...,뉴스1
5,"문 대통령, 코로나19 현장 종사자 등 1만 5천명에 설 선물",2022.01.18. 오전 11:13,문재인 대통령. 〈사진-연합뉴스〉 문재인 대통령이 설 명절을 맞아 코...,https://news.naver.com/main/read.naver?mode=LS...,JTBC
6,"모더나 CEO ""독감·코로나 한번에 예방하는 백신 개발""",2022.01.18. 오전 11:29,"""내년 하반기까지 개발 목표""""두세 번 맞는 번거로움 없애겠다""미국 제약회사 모더나...",https://news.naver.com/main/read.naver?mode=LS...,한경닷컴
7,"광주시, 코로나19 예방접종 피해보상 신속 추진",2022.01.18. 오후 5:05,코로나19 백신 접종 현장. 방역당국 제공광주시가 코로나19 예방접종 이상 반응에 ...,https://news.naver.com/main/read.naver?mode=LS...,노컷뉴스
8,"코로나19 확진자 평균 치료비 308만원, 건보공단 부담 263만원",2022.01.18. 오후 4:59,강도태 건보공단 이사장 신년간담회 “하반기 건보료 부과체계 개편…국민 수용성↑”“코...,https://news.naver.com/main/read.naver?mode=LS...,The Internet Hankyoreh
9,"""홍콩서 동물-사람 간 코로나19 전염 의심 사례 나와""",2022.01.18. 오후 4:59,자료사진홍콩에서 동물과 사람 사이 코로나19 전염 의심 사례가 처음 나왔다고 홍콩 ...,https://news.naver.com/main/read.naver?mode=LS...,MBC
